# Load Package

In [1]:
%matplotlib inline
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
from scipy import stats
import theano.tensor as tt

In [2]:
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
from matplotlib.figure import Figure
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvas

# Model Preparation

## Test1

In [3]:
y_obs =  np.random.normal(0, 1, size=100)
idatas_cmp = {}

with pm.Model() as mA:
    σ = pm.HalfNormal("σ", 1)
    y = pm.SkewNormal("y", 0, σ, alpha=1, observed=y_obs)
    idataA = pm.sample(return_inferencedata=True)
    idataA.add_groups({"posterior_predictive": {"y":pm.sample_posterior_predictive(idataA)["y"][None,:]}})

with pm.Model() as mB:
    σ = pm.HalfNormal("σ", 1)
    y = pm.Normal("y", 0, σ, observed=y_obs)
    idataB = pm.sample(return_inferencedata=True)
    idataB.add_groups({"posterior_predictive": {"y":pm.sample_posterior_predictive(idataB)["y"][None,:]}})


with pm.Model() as mC:
    μ = pm.Normal("μ", 0, 1)
    σ = pm.HalfNormal("σ", 1)
    y = pm.Normal("y", μ, σ, observed=y_obs)
    idataC = pm.sample(return_inferencedata=True)
    idataC.add_groups({"posterior_predictive": {"y":pm.sample_posterior_predictive(idataC)["y"][None,:]}})

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.8797279616790327, but should be close to 0.8. Try to increase the number of tuning steps.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 12 seconds.
The acceptance probability does not match the target. It is 0.8845026142286556, but should be close to 0.8. Try to increase the number of tuning steps.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ, μ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 13 seconds.


In [4]:
idatas_cmp["mA"] = idataA
idatas_cmp["mB"] = idataB
idatas_cmp["mC"] = idataC

In [5]:
idatas_cmp

{'mA': Inference data with groups:
 	> posterior
 	> posterior_predictive
 	> log_likelihood
 	> sample_stats
 	> observed_data,
 'mB': Inference data with groups:
 	> posterior
 	> posterior_predictive
 	> log_likelihood
 	> sample_stats
 	> observed_data,
 'mC': Inference data with groups:
 	> posterior
 	> posterior_predictive
 	> log_likelihood
 	> sample_stats
 	> observed_data}

## Test 2

In [6]:
# idatas_cmp = {}
# d1 = az.load_arviz_data("centered_eight")
# d2 = az.load_arviz_data("non_centered_eight")
# idatas_cmp["mA"] = d1
# idatas_cmp["mB"] = d2

# Dashboard Building

In [7]:
pn.extension()

In [9]:
# Structure
model_select=pn.WidgetBox()
model_selection1 = pn.widgets.Select(value=None, options = list(idatas_cmp.keys()) , name="Model1")
model_selection2 = pn.widgets.Select(value=None, options = list(idatas_cmp.keys()) , name="Model2")
str_pane = pn.pane.Str('', style={'font-size': '12pt','color':'red'})


global idatas_cmp
global str_pane


#function
@pn.depends(model_selection1.param.value, model_selection2.param.value)
def get_compare_plot(model_selection1,model_selection2):
    if model_selection1!=model_selection2:
        str_pane.object=""
        keys=[]
        keys.append(model_selection1)
        keys.append(model_selection2)
        fig, (scatter_plt, bar_plot) = plt.subplots(1, 2)
        dict_cmp = {x:idatas_cmp[x] for x in keys}
        scatter_plt=az.plot_elpd(dict_cmp, plot_kwargs={"marker":"."}, threshold=2)
        return(scatter_plt.figure)
    else:
        str_pane.object='Please try again. You can only compare two different models'

@pn.depends(model_selection1.param.value, model_selection2.param.value)
def get_bar_plot(model_selection1,model_selection2):
    if model_selection1!=model_selection2:
        str_pane.object=""
        keys=[]
        keys.append(model_selection1)
        keys.append(model_selection2)
        
        dict_cmp = {x:idatas_cmp[x] for x in keys}
        scatter_plt=az.plot_elpd(dict_cmp, plot_kwargs={"marker":"."}, threshold=2)
        ax=scatter_plt.figure.get_axes()[0]
        x_ob=ax.collections[0].get_offsets()[:,0]
        y_ob=ax.collections[0].get_offsets()[:,1]
        pos=sum(1 for y_ob_point in list(y_ob) if y_ob_point < 0)
        neg=sum(1 for y_ob_point in list(y_ob) if y_ob_point > 0)
        bar_data = {}
        bar_data["positive"]=pos
        bar_data["negative"]=neg
        fig, ax = plt.subplots()
        ax.bar(list(bar_data.keys()),list(bar_data.values()),color ='maroon',width = 0.4)
        ax.set_xticks(list(bar_data.keys()))
        ax.set_title("Summary of Observation (Negative Vs Positive)")
        return(ax.figure)
    else:
        str_pane.object='Please try again. You can only compare two different models'

# show up
pn.Column(pn.Row(model_selection1,model_selection2),str_pane,pn.Row(get_compare_plot,get_bar_plot))

Column
    [0] Row
        [0] Select(name='Model1', options=['mA', 'mB', 'mC'], value='mA')
        [1] Select(name='Model2', options=['mA', 'mB', 'mC'], value='mA')
    [1] Str(str, style={'font-size': '12pt', ...})
    [2] Row
        [0] ParamFunction(function)
        [1] ParamFunction(function)